In [608]:
from bs4 import BeautifulSoup
import requests
import time, os 
import pandas as pd

In [595]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

## (1) extract movies from, 2019, 2020, 2021.

In [929]:
def extracts_fields(list_tags,year_release):

    movies_dict = {}
    for i in range(len(list_tags)): # link is a tag object now (unlike list_tags) 
        movie_name = list_tags[i].find('a').text
    
        movie_ID = list_tags[i].find('a')['href'].split("/")[2]
        imdb_movie_link = url_imdb_movie + list_tags[i].find('a')['href'].split("/")[2] + '?ref_=fn_al_tt_1'
       
        try:
            MPPA = list_tags[i].find(class_='certificate').text.strip()
        except: 
            MPPA = []
     
        try:
            runtime = list_tags[i].find(class_='runtime').text.strip().split(" ")[0]
        except:
            runtime = []
     
        try:
            genre = list_tags[i].find(class_='genre').text.strip().split(",")
        
        except:
            genre = []
        
        try: 
            imdb_rating = list_tags[i].find('strong').text.strip()
        except:
            imdb_rating = []
        
        try:
            votes = list_tags[i].find(class_='sort-num_votes-visible').find('span').findNext().text.strip().replace(',', '')
        except:
            votes = []

        try:
            metacritic = list_tags[i].find(class_='inline-block ratings-metascore').find('span').contents[0].strip()
        except: # In case there is no metacritic.
            metacritic = []
            
        if type(votes) == str:
            num_votes_last_video = int(votes)
        else:
            num_votes_last_video = votes
    
        movies_dict[movie_name] = [movie_ID]+[imdb_movie_link]+[MPPA]+[runtime]+[genre]+[imdb_rating]+\
        [votes]+[metacritic]+[year_release]
    
    return movies_dict,num_votes_last_video

In [930]:
import random
from fake_useragent import UserAgent
ua = UserAgent()


In [931]:
url_imdb_movie = 'https://www.imdb.com/title/'

In [932]:
def list_movies_year(year_release):

    imdb_query = f'https://www.imdb.com/search/title/?title_type=feature&release_date={year_release}-01-01,{year_release}-12-31&countries=us&languages=en&sort=num_votes,desc&count=250'
    user_agent = {'User-agent': ua.random} # generates a random user agent.
    page = requests.get(imdb_query,headers = user_agent).text
    soup = BeautifulSoup(page, 'html5lib')
    driver = webdriver.Chrome(chromedriver) # Web driver object.
    driver.get(imdb_query) # like navigating to this query on the browserb

    # Define headers
    headers_movies = ['movie_ID', 'link',
               'MPPA', 'runtime', 'genre','IMDB_rating','number_of_votes','metacritic','year_release']

    num_votes_last_video = 9999999 # initialize value, bigger than 1000.
    num_page = 1
    while num_votes_last_video > 200: 
        if num_page > 1: # move to the next page.
            filter_button = driver.find_element_by_xpath(
            '//a[contains(@class, "lister-page-next next-page")]')
            filter_button.click()

        soup = BeautifulSoup(driver.page_source, 'html.parser') # parse current page.
        list_tags = soup.find_all(class_="lister-item-content") # find the list tags.
        movies_dict,num_votes_last_video = extracts_fields(list_tags,year_release) # extract the movie information of interest.

        movies_current_page = pd.DataFrame(movies_dict).T
        movies_current_page.columns = headers_movies

        if num_page == 1:
            movies_list = movies_current_page
        else:
            movies_list = pd.concat([movies_list,movies_current_page],axis = 0)

        del movies_dict,movies_current_page

        time.sleep(.5+40*random.random()) # random time between requests.

        movies_list.to_csv(f'movies_list_{year_release}_until_page{num_page}.csv') # save for safety
        num_page += 1
        
    driver.close()
    
    return movies_list



In [933]:
movies_2019 = list_movies_year(2019)
time.sleep(320)
movies_2020 = list_movies_year(2020)
time.sleep(320)
movies_2021 = list_movies_year(2021)


<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, Zack Snyder's Justice League to Zone Drifter
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_ID         500 non-null    object
 1   link             500 non-null    object
 2   MPPA             500 non-null    object
 3   runtime          500 non-null    object
 4   genre            500 non-null    object
 5   IMDB_rating      500 non-null    object
 6   number_of_votes  500 non-null    object
 7   metacritic       500 non-null    object
 8   year_release     500 non-null    object
dtypes: object(9)
memory usage: 39.1+ KB


Concatenate the three data frames:

In [1237]:
movies_links = pd.concat([movies_2019,movies_2020,movies_2021],axis = 0) 
movies_links.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2249 entries, Joker to Zone Drifter
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_ID         2249 non-null   object
 1   link             2249 non-null   object
 2   MPPA             2249 non-null   object
 3   runtime          2249 non-null   object
 4   genre            2249 non-null   object
 5   IMDB_rating      2249 non-null   object
 6   number_of_votes  2249 non-null   object
 7   metacritic       2249 non-null   object
 8   year_release     2249 non-null   object
dtypes: object(9)
memory usage: 175.7+ KB


In [1226]:
movies_links.tail()

,movie_ID,link,MPPA,runtime,genre,IMDB_rating,number_of_votes,metacritic,year_release
Star-Crossed: The Film,tt15285682,https://www.imdb.com/title/tt15285682?ref_=fn_...,TV-G,48,[Music],6.6,102,[],2021
The Connection,tt9826820,https://www.imdb.com/title/tt9826820?ref_=fn_a...,[],86,[Sci-Fi],3.0,101,[],2021
This Is the Night,tt7743120,https://www.imdb.com/title/tt7743120?ref_=fn_a...,R,107,"[Comedy, Drama]",5.7,101,[],2021
It Takes Three,tt7539904,https://www.imdb.com/title/tt7539904?ref_=fn_a...,Not Rated,90,"[Comedy, Romance]",5.8,100,[],2021
Zone Drifter,tt14730740,https://www.imdb.com/title/tt14730740?ref_=fn_...,[],93,[Sci-Fi],2.7,100,[],2021


In [1238]:
movies_links['number_of_votes'] = pd.to_numeric(movies_links['number_of_votes']) # it was a string!
movies_links['year_release'] = pd.to_numeric(movies_links['year_release']) # it was a string!


movies_links.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2249 entries, Joker to Zone Drifter
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_ID         2249 non-null   object
 1   link             2249 non-null   object
 2   MPPA             2249 non-null   object
 3   runtime          2249 non-null   object
 4   genre            2249 non-null   object
 5   IMDB_rating      2249 non-null   object
 6   number_of_votes  2249 non-null   int64 
 7   metacritic       2249 non-null   object
 8   year_release     2249 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 175.7+ KB


In [1239]:
len(movies_links[movies_links['number_of_votes'] >= 200]) # We will have 1930 pages.

1930

In [1240]:
movies_links = movies_links.reset_index()
movies_links = movies_links.rename(columns = {'index':'title'})
movies_links

,title,movie_ID,link,MPPA,runtime,genre,IMDB_rating,number_of_votes,metacritic,year_release
0,Joker,tt7286456,https://www.imdb.com/title/tt7286456?ref_=fn_a...,R,122,"[Crime, Drama, Thriller]",8.4,1067788,59,2019
1,Avengers: Endgame,tt4154796,https://www.imdb.com/title/tt4154796?ref_=fn_a...,PG-13,181,"[Action, Adventure, Drama]",8.4,939831,78,2019
2,Once Upon a Time... In Hollywood,tt7131622,https://www.imdb.com/title/tt7131622?ref_=fn_a...,R,161,"[Comedy, Drama]",7.6,637902,83,2019
3,Knives Out,tt8946378,https://www.imdb.com/title/tt8946378?ref_=fn_a...,PG-13,130,"[Comedy, Crime, Drama]",7.9,528298,82,2019
4,Captain Marvel,tt4154664,https://www.imdb.com/title/tt4154664?ref_=fn_a...,PG-13,123,"[Action, Adventure, Sci-Fi]",6.8,491971,64,2019
...,...,...,...,...,...,...,...,...,...,...
2244,Star-Crossed: The Film,tt15285682,https://www.imdb.com/title/tt15285682?ref_=fn_...,TV-G,48,[Music],6.6,102,[],2021
2245,The Connection,tt9826820,https://www.imdb.com/title/tt9826820?ref_=fn_a...,[],86,[Sci-Fi],3.0,101,[],2021
2246,This Is the Night,tt7743120,https://www.imdb.com/title/tt7743120?ref_=fn_a...,R,107,"[Comedy, Drama]",5.7,101,[],2021
2247,It Takes Three,tt7539904,https://www.imdb.com/title/tt7539904?ref_=fn_a...,Not Rated,90,"[Comedy, Romance]",5.8,100,[],2021


# And now, web-scrapping on each movie page:

In [1264]:
def total_awards_extractor(condition):
    awards_info = soup.find('span',class_='ipc-metadata-list-item__list-content-item').text
    if condition in awards_info:
        for type_award in awards_info.split('&'): # split to separate between wins and nominations.
            if condition in type_award:
                return int(type_award.strip().split(' ')[0])
        

In [1265]:
def scraps_movie_ID_page(soup,title):

    movie_page = {} 
    
    num_user_reviews = 0
    num_critic_reviews = 0
    total_awards_wins = 0
    total_awards_nominations = 0
    oscar_wins = 0
    oscar_nominations = 0

    # Release date and Country:
    try:
        release_info = soup.find(text=re.compile('Release date')).findNext().text.split('(')
        release_date = release_info[0].strip() 
        release_country = release_info[1].split(')')[0]

    except:
        release_date = []
        release_country = []

    # number user reviews: 
    try:
        for i in range(len(soup.find_all(class_="label"))):
            if soup.find_all(class_="label")[i].text == 'User reviews':
                num_user_reviews = soup.find_all(class_="label")[i].findPrevious().text
    except:  
        num_user_reviews = 0

    # Number of critic reviews:    
    try:
        num_critic_reviews = soup.find(text=re.compile('Critic reviews')).findPrevious().findPrevious().text
    except: 
        num_critic_reviews = 0


    # Total awards and nominations:
    try:
        total_awards_wins = total_awards_extractor('win')
    except:  
        total_awards_wins = 0
    try:
        total_awards_nominations = total_awards_extractor('nomination')
    except:
        total_awards_nominations = 0


    # Total Oscar Wins:
    try:
        span_candidates = soup.find_all('a',class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link")
        for span_candidate in span_candidates:
            if "Won" in span_candidate.text: # or "Nominated" in span_candidate.text
                oscar_wins = span_candidate.text.split(' ')[1]
    except:
        oscar_wins = 0
    
    # Total Oscar Nominations (only reported if there were no wins):
    try:
        span_candidates = soup.find_all('a',class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link")
        for span_candidate in span_candidates:
            if "Nominated" in span_candidate.text: 
                oscar_nominations = span_candidate.text.split(' ')[2]     
    except:
        oscar_nominations = 0    

    # Production companies:
    try:
        classes_repetitions = soup.find_all(class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link")
        production_companies = []
        for class_repetition in classes_repetitions:
            if class_repetition.text == 'Production companies':
                companies = class_repetition.findNext().find_all(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
                for current_company in companies:
                    if current_company.text not in production_companies:
                        production_companies.append(current_company.text)  
    except:  
        production_companies = []

    # Box office: 
    try:
        budget = soup.find(text=re.compile('Budget')).findNext().text.replace(',','').split(' ')[0]
    except:
        budget = []
    try:    
        opening_weekend_usa_can = soup.find(text=re.compile('Opening weekend US & Canada')).findNext().findNext().\
            findNext().text.replace(',','')
    except:
        opening_weekend_usa_can = []

    try:
        gross_usa_can = soup.find(text=re.compile('Gross US & Canada')).findNext().findNext().findNext().text.replace(',','')
    except:    
        gross_usa_can = []

    try:    
        gross_worldwide = soup.find(text=re.compile('Gross worldwide')).findNext().text.replace(',','')
    except:
        gross_worldwide = []

    # Director
    try:
        director = soup.find(text=re.compile('Director')).findNext().findNext().findNext().text
    except: 
        director = []
    # Top actor:
    try:
        top_star = soup.find(text=re.compile('Stars')).findNext().findNext().findNext().findNext().text
    except:   
        top_star = []        
    # Top writer:
    try:
        top_writer = soup.find(text=re.compile('Writers')).findNext().findNext().findNext().findNext().text
    except:   
        top_writer = []
        
    movie_name 
    movie_page[movie_name] = [title]+[movie_ID]+[release_date]+[release_country]+[num_user_reviews]+[num_critic_reviews]+\
    [total_awards_wins]+[total_awards_nominations]+[oscar_wins]+[oscar_nominations]+[production_companies]+[budget]+[opening_weekend_usa_can]+\
    [gross_usa_can]+[gross_worldwide]+[director]+[top_star]+[top_writer]
    
    
    time.sleep(.5+40*random.random()) # random time between requests.
    

    return movie_page 

In [1266]:
# Define headers
headers_movies = ['title','movie_ID','release_date','release_country','num_user_reviews','num_critic_reviews',\
                  'total_awards_wins','total_nominations',
                  'oscar_wins','oscar_nominations','production_companies','budget','opening_weekend_usa_can',\
                  'gross_usa_can','gross_worldwide', 'director','top_star','top_writer']

for i in range(len(movies_database['link']):
    
    if movies_links['number_of_votes'][i] >= 200: 
        link_imdb = movies_links['link'][i]        
        user_agent = {'User-agent': ua.random} # generates a random user agent.
        page = requests.get(link_imdb,headers = user_agent).text
        soup = BeautifulSoup(page, 'html5lib')
        driver = webdriver.Chrome(chromedriver) # Web driver object.
        driver.get(link_imdb) # like navigating to this query on the browser


        soup = BeautifulSoup(driver.page_source, 'html.parser') # parse current page.
        current_movie_dict = scraps_movie_ID_page(soup,movies_links['title'][i])
            
        current_movie_df = pd.DataFrame(current_movie_dict).T
        current_movie_df.columns = headers_movies

        if i == 0:
            movie_df_2 = current_movie_df
        else:
            movie_df_2 = pd.concat([movie_df_2,current_movie_df],axis = 0)

        del current_movie_dict,current_movie_df  
        
        
        if (i+1 % 10 == 0):
            movie_df_2.to_csv(f'movies_database2_until_movie_{i}.csv') # save for safety               
            
        driver.close()
        
        if (i+1 % 150 == 0):
            time.sleep(320)
            
            
     



In [1267]:
movie_df_2.head()

,title,movie_ID,release_date,release_country,num_user_reviews,num_critic_reviews,total_awards_wins,total_nominations,oscar_wins,oscar_nominations,production_companies,budget,opening_weekend_usa_can,gross_usa_can,gross_worldwide,director,top_star,top_writer
Booksmart,Joker,tt1489887,"October 4, 2019",United States,11K,743,118,239,2,0,"[Warner Bros., Village Roadshow Pictures]",$55000000,$96202337,$335451311,$1074419384,Todd Phillips,Joaquin Phoenix,Todd Phillips
Booksmart,Avengers: Endgame,tt1489887,"April 26, 2019",United States,9.3K,583,70,122,0,1,"[Marvel Studios, Walt Disney Pictures]",$356000000,$357115007,$858373000,$2797501328,Anthony Russo,Robert Downey Jr.,Christopher Markus
